<a href="https://colab.research.google.com/github/sbooeshaghi/intervals/blob/master/notebooks/bustools_merge_sim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random

In [2]:
random.seed(16)

In [3]:
def distributed_sum(val, n):
  ret = []

  remain = val%n
  split = val//n
  ret = [split]*n

  for i in range(remain):
    ret[i] += 1

  return ret

In [8]:
def gen_itv(minimum, maximum, min_len, max_len):
  lower = random.randint(minimum, maximum-1)
  upper = random.randint(lower, maximum)
  while upper - lower < min_len or upper - lower > max_len:
    upper = random.randint(lower, maximum)

  return (lower, upper)

def gen_itvs(minimum, maximum, min_len, max_len, n):
  itvs = []
  for i in range(n):
    itvs.append(gen_itv(minimum, maximum, min_len, max_len))
  return itvs

def remove_overlap(itvs):
  srt = sorted(itvs, key=lambda x: x[0])
  prev = srt[0]
  new = []
  for s in srt[1:]:
    if prev[1] <= s[0]:
      new.append(prev)
      prev = s
  new.append(prev)
  return new

In [9]:
nf = 8
N = 100 # bus records
N_ec = 1000 # number of ecs

max_itvs = 31 # each kmer had a diff ec

rpf = distributed_sum(N, nf)

# intervals
n_itvs = 5 # must be less than 26 only because we use letters of the english alphabet to label each interval
max_pos = 61
min_pos = 0

min_len = 1
max_len = max_pos

In [10]:
f = []

for r in rpf:
  bd = sorted(random.sample(range(N), r))
  tmp = []
  for ridx in range(r):
    intervals = gen_itvs(min_pos, max_pos, min_len, max_len, random.randint(1, n_itvs))
    unique = remove_overlap(intervals)
    ec = random.sample(range(N_ec), len(unique))
    for i, e in zip(unique, ec):
      tmp.append((bd[ridx], e, i[0]))
      tmp.append((bd[ridx], e, i[1]))
  f.append(tmp)

# Write data to text files

In [ ]:
import contextlib

In [25]:
filenames = [f"{i}.txt" for i in range(nf)]

In [30]:
with contextlib.ExitStack() as stack:
    files = [stack.enter_context(open(fname, 'w')) for fname in filenames]
    for outf, fdata in zip(files, f):
      for d in fdata:
        outf.write(f"{d[0]}\t{d[1]}\t{d[2]}\n")

# Read in files

In [32]:
import heapq

In [102]:
import os

In [149]:
pq = []
with contextlib.ExitStack() as stack:
    files = [stack.enter_context(open(fname, 'r')) for fname in filenames]
    for fidx, fin in enumerate(files):
      l = tuple(map(int, fin.readline().strip().split('\t')))
      r = l[0]
      e = l[1]
      i = l[2]
      rold = r
      # this reads in a single read
      while rold == l[0]:

        heapq.heappush(pq, (r, (e, i), fidx) )
        rin = fin.readline()

        l = tuple(map(int, rin.strip().split('\t')))
        r = l[0]
        e = l[1]
        i = l[2]
      fin.seek(fin.tell() - len(rin), os.SEEK_SET)

    #######
    prev = heapq.heappop(pq)
    while len(pq)>0:
      readfrom = set()
      curr = heapq.heappop(pq)
      while prev[0] == curr[0] and len(pq)>0:
        readfrom.add(prev[-1])
        curr = heapq.heappop(pq)
        
        prev = curr
        print(curr)
      print("-----------------")

      for file_idx in readfrom:
        if rin ==  '':
            break
        l = tuple(map(int, files[file_idx].readline().strip().split('\t')))
        r = l[0]
        e = l[1]
        i = l[2]
        rold = r
        # this reads in a single read
        while rold == l[0]:

          heapq.heappush(pq, (r, (e, i), file_idx) )

          rin = files[file_idx].readline()
          if rin ==  '':
            break

          l = tuple(map(int, rin.strip().split('\t')))
          r = l[0]
          e = l[1]
          i = l[2]
        files[file_idx].seek(files[file_idx].tell() - len(rin), os.SEEK_SET)


      


(0, (789, 27), 2)
(0, (789, 52), 2)
(1, (51, 51), 0)
(1, (51, 52), 0)
(1, (405, 10), 0)
(1, (405, 42), 0)
(3, (15, 16), 1)
(3, (15, 46), 1)
(3, (305, 51), 1)
(3, (305, 57), 1)
(3, (843, 0), 7)
(3, (843, 35), 7)
(3, (910, 46), 7)
(3, (910, 52), 7)
(7, (289, 25), 4)
(7, (289, 39), 4)
(7, (500, 60), 4)
(7, (500, 61), 4)
(10, (183, 57), 5)
(10, (183, 60), 5)
(13, (86, 46), 6)
(13, (86, 61), 6)
(13, (436, 23), 6)
(13, (436, 39), 6)
(18, (444, 0), 3)
(18, (444, 55), 3)
(18, (692, 60), 3)
(18, (692, 61), 3)
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------


''